In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame

In [2]:
STable1: DataFrame = pd.read_csv("STable1_all_genomes_info.tsv", sep="|")
STable2: DataFrame = pd.read_csv("STable2_BiG-SLICE_t0.4_GCF_assignment.csv", sep=",")
STable3: DataFrame = pd.read_csv("STable3_BiG-SLICE_t0.5_GCF_assignment.csv", sep=",")
STable4: DataFrame = pd.read_csv("STable4_BiG-SLICE_t0.6_GCF_assignment.csv", sep=",")
STable5: DataFrame = pd.read_csv("STable5_BiG-SLICE_t0.7_GCF_assignment.csv", sep=",")

In [3]:
thresholds_4_gcfs: np.ndarray = np.array(STable2.gcf_id.unique())
thresholds_5_gcfs: np.ndarray = np.array(STable3.gcf_id.unique())
thresholds_6_gcfs: np.ndarray = np.array(STable4.gcf_id.unique())
thresholds_7_gcfs: np.ndarray = np.array(STable5.gcf_id.unique())

In [4]:
bgc_ids: DataFrame = STable1.bgc_ids.str.split(",", expand=True)
bgc_ids = (bgc_ids.apply(pd.to_numeric, downcast="unsigned"))
bgc_ids[["dataset_name", "AccNo", "taxonomy"]] = STable1[["dataset_name", "AccNo", "taxonomy"]]

In [5]:
STable2["bgc_id"] = STable2["bgc_id"].astype(np.int64)

In [6]:
STable2.index = STable2["bgc_id"]
STable2["taxonomy"] = ""

In [7]:
from functools import lru_cache
from pandarallel import pandarallel
import os
pandarallel.initialize(nb_workers=os.cpu_count())
def add_taxonomy_to_table(gcfs_array: np.ndarray, stable: DataFrame) -> None:
    # print(type(gcfs_array))
    # print(gcfs_array[5])
    for gcf in gcfs_array:
        # print(f"gcf{gcf}\n")
        bgc_ids_of_gcf: np.ndarray = np.array(STable2[STable2.gcf_id == gcf]["bgc_id"])
        for bgc_id in bgc_ids_of_gcf:
            # print(f"bgc_id{bgc_id}\n")
            # print(f"type(bgc_id{type(bgc_id)}\n")
            bgc_tax = bgc_ids[bgc_ids.eq(bgc_id).any(axis="columns")]["taxonomy"].iloc[0]
            stable.loc[bgc_id, "taxonomy"] = bgc_tax

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [8]:
#add_taxonomy_to_table = np.vectorize(add_taxonomy_to_table)

add_taxonomy_to_table(gcfs_array = thresholds_4_gcfs[-10:], stable = STable2)

In [9]:
STable2.sort_values("taxonomy",ascending=False)

,bgc_id,gcf_id,feature_to_centroid_distance,taxonomy
bgc_id,,,,
1318457,1318457,73245,0.000000,"Bacteria,Proteobacteria,Alphaproteobacteria,Sp..."
1318445,1318445,74585,0.000000,"Bacteria,Proteobacteria,Alphaproteobacteria,Az..."
1318508,1318508,68629,0.000000,"Bacteria,Planctomycetota,Planctomycetes,Pirell..."
1318428,1318428,69139,0.000000,"Bacteria,Firmicutes,Bacilli,Lactobacillales,Ae..."
1318426,1318426,70750,0.000000,"Bacteria,Firmicutes,Bacilli,Lactobacillales,Ae..."
...,...,...,...,...
399406,399406,11705,0.007830,
399405,399405,39830,0.031746,
399404,399404,43972,0.018364,


# END OF ADDITIONAL EFFORTS